In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

CPU times: user 23.4 s, sys: 267 ms, total: 23.7 s
Wall time: 24.7 s


In [5]:
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
prefixes = ["ICD10CM", "MONDO"]
condition_1 = msdf_mondo.df['object_id'].str.contains("|".join(prefixes))
condition_2 = msdf_mondo.df['subject_id'].str.contains("|".join(prefixes))
msdf_mondo.df = msdf_mondo.df[condition_1 & condition_2]
msdf_mondo.df

,subject_id,subject_label,predicate_id,object_id,mapping_justification
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified
...,...,...,...,...,...
71865,MONDO:0100347,carcinoid syndrome,skos:exactMatch,ICD10CM:E34.0,Unspecified
71939,MONDO:0100471,vitamin D deficiency,skos:exactMatch,ICD10CM:E55,Unspecified
71954,MONDO:0100491,generalized pustular psoriasis,skos:exactMatch,ICD10CM:L40.1,Unspecified
72050,MONDO:0800029,interstitial lung disease 2,skos:exactMatch,ICD10CM:J84.112,Unspecified


In [6]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = msdf_lex.df['subject_id'].str.contains("|".join(prefixes))
condition_2 = msdf_lex.df['object_id'].str.contains("|".join(prefixes))
msdf_lex.df = msdf_lex.df[condition_1 & condition_2]
msdf_lex.df

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
28311,ICD10CM:A07.3,Isosporiasis,skos:exactMatch,MONDO:0018769,isosporiasis,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,Isosporosis
28321,ICD10CM:A08.3,Other viral enteritis,skos:exactMatch,ICD10CM:A08.39,Other viral enteritis,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,Other viral enteritis
28326,ICD10CM:A15.0,Tuberculosis of lung,skos:closeMatch,MONDO:0021953,tuberculous fibrosis of lung,semapv:LexicalMatching,oaklib,0.500000,oio:hasExactSynonym,oio:hasRelatedSynonym,Tuberculous fibrosis of lung
28331,ICD10CM:A17.8,Other tuberculosis of nervous system,skos:exactMatch,ICD10CM:A17.89,Other tuberculosis of nervous system,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,Other tuberculosis of nervous system
28346,ICD10CM:A19,Miliary tuberculosis,skos:exactMatch,MONDO:0005848,miliary tuberculosis,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,disseminated tuberculosis
...,...,...,...,...,...,...,...,...,...,...,...
41091,MONDO:0800026,"central hypoventilation syndrome, congenital, ...",skos:closeMatch,MONDO:0800031,"central hypoventilation syndrome, congenital",semapv:LexicalMatching,oaklib,0.500000,oio:hasRelatedSynonym,rdfs:label,"central hypoventilation syndrome, congenital"
41092,MONDO:8000010,antiphospholipid syndrome,skos:broadMatch,MONDO:8000014,familial antiphospholipid syndrome,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasBroadSynonym,Hughes syndrome
41093,MONDO:8000015,"46,XY sex reversal 11",skos:closeMatch,MONDO:8000017,testicular regression syndrome,semapv:LexicalMatching,oaklib,0.500000,oio:hasRelatedSynonym,oio:hasExactSynonym,TRS
41094,MONDO:8000015,"46,XY sex reversal 11",skos:closeMatch,MONDO:8000017,testicular regression syndrome,semapv:LexicalMatching,oaklib,0.500000,oio:hasRelatedSynonym,rdfs:label,testicular regression syndrome


In [7]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
comparison_df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment,object_label,mapping_tool,confidence,subject_match_field,object_match_field,match_string
67784,MONDO:0024647,urolithiasis,skos:exactMatch,ICD10CM:N20-N23,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
14794,MONDO:0005133,endometriosis,skos:exactMatch,ICD10CM:N80,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
60838,MONDO:0019095,plague,skos:broadMatch,ICD10CM:A20.8,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
66084,MONDO:0021465,benign neoplasm of appendix,skos:exactMatch,ICD10CM:D12.1,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
17330,MONDO:0005895,paragonimiasis,skos:exactMatch,ICD10CM:B66.4,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains("|".join(mappings)))
]

unmapped_df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment,object_label,mapping_tool,confidence,subject_match_field,object_match_field,match_string
67784,MONDO:0024647,urolithiasis,skos:exactMatch,ICD10CM:N20-N23,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
14794,MONDO:0005133,endometriosis,skos:exactMatch,ICD10CM:N80,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
60838,MONDO:0019095,plague,skos:broadMatch,ICD10CM:A20.8,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
66084,MONDO:0021465,benign neoplasm of appendix,skos:exactMatch,ICD10CM:D12.1,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
17330,MONDO:0005895,paragonimiasis,skos:exactMatch,ICD10CM:B66.4,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Let reviewers check if this makes sense or no.
unmapped_lex_exact = unmapped_df[(unmapped_df['comment'] == 'LEXMATCH') & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_lex_exact.to_csv(join(dir_name, "unmapped_lex_exact.tsv"), sep='\t', index = False)
unmapped_lex_exact.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment,object_label,mapping_tool,confidence,subject_match_field,object_match_field,match_string
34948,ICD10CM:R26.8,Other abnormalities of gait and mobility,skos:exactMatch,ICD10CM:R26.89,semapv:LexicalMatching,LEXMATCH,Other abnormalities of gait and mobility,oaklib,0.849779,rdfs:label,rdfs:label,Other abnormalities of gait and mobility
30985,ICD10CM:H50.61,Brown's sheath syndrome,skos:exactMatch,MONDO:0014624,semapv:LexicalMatching,LEXMATCH,Brown syndrome,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,Brown's sheath syndrome
30529,ICD10CM:G45.9,"Transient cerebral ischemic attack, unspecified",skos:exactMatch,MONDO:0005264,semapv:LexicalMatching,LEXMATCH,transient ischemic attack,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,TIA
39794,MONDO:0010758,Wieacker-Wolff syndrome,skos:exactMatch,MONDO:0019024,semapv:LexicalMatching,LEXMATCH,mast cell sarcoma,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,MCS
29750,ICD10CM:E16.4,Increased secretion of gastrin,skos:exactMatch,MONDO:0019610,semapv:LexicalMatching,LEXMATCH,Zollinger-Ellison syndrome,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,Zollinger-Ellison syndrome


In [10]:
# Inspect why these are missing from SSSOM mappings
unmapped_mondo_exact = unmapped_df[(unmapped_df['comment'] == 'MONDO_MAPPINGS')& (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
unmapped_mondo_exact.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment,object_label,mapping_tool,confidence,subject_match_field,object_match_field,match_string
67784,MONDO:0024647,urolithiasis,skos:exactMatch,ICD10CM:N20-N23,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
14794,MONDO:0005133,endometriosis,skos:exactMatch,ICD10CM:N80,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
66084,MONDO:0021465,benign neoplasm of appendix,skos:exactMatch,ICD10CM:D12.1,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
17330,MONDO:0005895,paragonimiasis,skos:exactMatch,ICD10CM:B66.4,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
66057,MONDO:0021459,benign neoplasm of esophagus,skos:exactMatch,ICD10CM:D13.0,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)
combined_msdf.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment,object_label,mapping_tool,confidence,subject_match_field,object_match_field,match_string
67784,MONDO:0024647,urolithiasis,skos:exactMatch,ICD10CM:N20-N23,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
14794,MONDO:0005133,endometriosis,skos:exactMatch,ICD10CM:N80,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
60838,MONDO:0019095,plague,skos:broadMatch,ICD10CM:A20.8,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
66084,MONDO:0021465,benign neoplasm of appendix,skos:exactMatch,ICD10CM:D12.1,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN
17330,MONDO:0005895,paragonimiasis,skos:exactMatch,ICD10CM:B66.4,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
%%time
df_dict = split_dataframe(combined_msdf)

CPU times: user 2.79 s, sys: 37.3 ms, total: 2.83 s
Wall time: 5.6 s


In [13]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

['mondo_exactmatch_icd10cm',
 'mondo_broadmatch_icd10cm',
 'icd10cm_narrowmatch_mondo',
 'icd10cm_closematch_mondo',
 'icd10cm_exactmatch_mondo',
 'icd10cm_broadmatch_mondo']

In [14]:
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [15]:
df_dict['mondo_exactmatch_icd10cm'].df

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment
0,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,MONDO_MAPPINGS
1,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,MONDO_MAPPINGS
2,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,MONDO_MAPPINGS
3,MONDO:0000245,tinea imbricata,skos:exactMatch,ICD10CM:B35.5,Unspecified,MONDO_MAPPINGS
4,MONDO:0000261,adenoiditis,skos:exactMatch,ICD10CM:J35.02,Unspecified,MONDO_MAPPINGS
...,...,...,...,...,...,...
1130,MONDO:0100347,carcinoid syndrome,skos:exactMatch,ICD10CM:E34.0,Unspecified,MONDO_MAPPINGS
1131,MONDO:0100471,vitamin D deficiency,skos:exactMatch,ICD10CM:E55,Unspecified,MONDO_MAPPINGS
1132,MONDO:0100491,generalized pustular psoriasis,skos:exactMatch,ICD10CM:L40.1,Unspecified,MONDO_MAPPINGS
1133,MONDO:0800029,interstitial lung disease 2,skos:exactMatch,ICD10CM:J84.112,Unspecified,MONDO_MAPPINGS
